In [16]:
import librosa
import numpy as np
import os
import pickle

g=os.listdir("wav_files")

spectrals=[]
for vidfile in g:

    dr=librosa.get_duration(path=f"wav_files/{vidfile}")
    y,sr = librosa.load(f"wav_files/{vidfile}",sr=10000/dr)



    #25k  points of sampled time serie data
    #print(f"Sampling data: {y.shape}")
    print(f"Sample rate: {sr}")
    mfccs = librosa.feature.mfcc(y=y, sr=sr)

    #print(mfccs.shape)

    mfccs_mean = np.mean(mfccs, axis=0)[::2]
    mfccs_max = np.max(mfccs, axis=0)[::2]

    #print(mfccs_max.shape)
    #print(mfccs_mean.shape)


    zero_crossings = librosa.zero_crossings(y, pad=False)
    #print(np.array((sum(zero_crossings))))
    song_characteristics=np.concatenate((mfccs_mean,mfccs_max))
    song_characteristics=np.concatenate((song_characteristics,[sum(zero_crossings)]))
    spectrals.append(song_characteristics)
    #https://towardsdatascience.com/extract-features-of-music-75a3f9bc265dnp.array(sum(zero_crossings))
spectrals=np.array(spectrals)
print(spectrals.shape)


Sample rate: 43.64248707944872
Sample rate: 36.82778027193432
Sample rate: 47.49272855094839
Sample rate: 43.256735887906785
Sample rate: 31.421571756894792
Sample rate: 42.58940491495253
Sample rate: 35.67462413021869
Sample rate: 33.60887018105197
Sample rate: 37.61915290880503
Sample rate: 42.54732883817427
Sample rate: 38.247252442273535
Sample rate: 40.4151710304054
Sample rate: 20.554794888316152
Sample rate: 41.1882232689365
Sample rate: 34.4421035268714
Sample rate: 34.74778622720671
Sample rate: 35.06465254030288
Sample rate: 51.49761125819524
Sample rate: 54.29451115733737
Sample rate: 36.17809664818549
Sample rate: 83.1912228543833
Sample rate: 54.954092771480695
Sample rate: 41.38612939650202
(23, 21)


In [17]:
import sklearn
#Mean, max, zero
print(spectrals[0])
spectrals_n=sklearn.preprocessing.normalize(spectrals,axis=0)
print(spectrals_n[0])


[-2.03211193e+01 -1.33304930e+01 -9.60407543e+00 -1.01035595e+01
 -1.06438961e+01 -9.52898598e+00 -6.66789770e+00 -8.98919106e+00
 -9.01910877e+00 -7.66273499e+00  3.31592560e+01  2.19552422e+01
  2.09942818e+01  2.07664165e+01  2.05237255e+01  2.20909348e+01
  2.09255714e+01  2.16866570e+01  2.31319237e+01  2.29226036e+01
  6.76000000e+03]
[-0.34777754 -0.43126741 -0.35646154 -0.38269578 -0.40256527 -0.36297935
 -0.24622564 -0.32132706 -0.3269315  -0.19674809  0.20927317  0.14136921
  0.12371461  0.11704895  0.11930552  0.12851114  0.13450513  0.13138251
  0.13389115  0.15637     0.22128283]


In [24]:
sklearn.cluster.KMeans()

AttributeError: module 'sklearn' has no attribute 'cluster'